In [1]:
# reformat (tall->wide) 7 PAH EPR 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, time

## Reformat EPR data

In [2]:
complete_input_file_path = '/Users/kimd999/research/projects/toxicity/per_each_data/7_PAH/01_11_2021/input/tall/7_PAH_zf_EPR_data_2021JAN11.csv'
behav_all_data = pd.read_csv(complete_input_file_path, header = 0)

In [3]:
behav_all_data.head()

,chemical.id,bottle.id,conc,plate.id,well,variable,value
0,3762,C20062,0.0,21269,H01,t1,0.0
1,3762,C20062,0.0,21269,H02,t1,0.0
2,3762,C20062,0.0,21269,H03,t1,0.0
3,3762,C20062,0.0,21269,H04,t1,0.0
4,3762,C20062,0.0,21269,H05,t1,0.0


In [4]:
# (Lisa) “A “NA” in the behavior data is that the animals that we move because they are dead, 
# or exhibit malformations that preclude them from having a “typical” LPR response.  
#So they should be excluded.”

behav_all_data['value'] = behav_all_data['value'].fillna(0)
#display(len(behav_all_data))

In [5]:
behav_all_data = behav_all_data.dropna()
display(len(behav_all_data))

100800

In [6]:
#behav_all_data_TX = behav_all_data.loc[behav_all_data['bottle.id'] == "TX002271",['chemical.id', 'bottle.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']]
#behav_all_data_TX.head()

In [7]:
# Keep only relevant columns
behav_all_data = behav_all_data.rename(columns = {"variable":"endpoint"})
columns_to_keep = ['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']
behav_all_data_select = behav_all_data.loc[:,columns_to_keep]
behav_all_data_select.head()

,chemical.id,conc,plate.id,well,endpoint,value
0,3762,0.0,21269,H01,t1,0.0
1,3762,0.0,21269,H02,t1,0.0
2,3762,0.0,21269,H03,t1,0.0
3,3762,0.0,21269,H04,t1,0.0
4,3762,0.0,21269,H05,t1,0.0


In [8]:
display(len(np.unique(behav_all_data_select['chemical.id'])))
display(np.unique(behav_all_data_select['endpoint']))
display(len(np.unique(behav_all_data_select['endpoint'])))

7

array(['t1', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17',
       't18', 't19', 't2', 't20', 't21', 't22', 't23', 't24', 't25',
       't26', 't27', 't28', 't29', 't3', 't30', 't31', 't32', 't33',
       't34', 't35', 't36', 't37', 't38', 't39', 't4', 't40', 't41',
       't42', 't43', 't44', 't45', 't46', 't47', 't48', 't49', 't5',
       't50', 't6', 't7', 't8', 't9'], dtype=object)

50

In [9]:
start_time = time.time()

reformatted_data = pd.DataFrame()
max_time = 50

full_devel = "full"
#full_devel = "devel"

#report = True
report = False

if (full_devel == "full"):
    chemical_id_from_here = np.unique(behav_all_data['chemical.id'])     # all chemicals
else: # full_devel = "devel"
    chemical_id_from_here = np.unique([3762])

for chemical_index in chemical_id_from_here:
    print("chemical_index:" + str(chemical_index))
    behav_data_chemical = behav_all_data_select.loc[behav_all_data['chemical.id'] == chemical_index,:]
    
    if (report): 
        display(behav_data_chemical.head())
        display(behav_data_chemical.tail())
        print ("unique_endpoint_per_chemical:" + str(unique_endpoint_per_chemical))
    
    unique_endpoint_per_chemical = np.unique(behav_data_chemical['endpoint'])

    # Append chemical_plate_well as a unique identifier
    behav_data_chemical.insert(0, 'chemical_plate_well', behav_data_chemical.loc[:,['chemical.id','plate.id', 'well']].apply(lambda x: '_'.join(x.map(str)), axis = 1))
    
    reported_len_group_endpoint = False
    for cpw in np.unique(behav_data_chemical.chemical_plate_well):
        temp_df = behav_data_chemical.loc[behav_data_chemical.chemical_plate_well == cpw,:]
        #display(temp_df.head())
        temp_df_grouped = temp_df.groupby(['chemical.id', 'plate.id', 'well'])
        # display(temp_df_grouped.head())
        for name, group in temp_df_grouped:
            if (reported_len_group_endpoint == False):
                print ("len(group.endpoint):" + str(len(group.endpoint)))
            reported_len_group_endpoint = True
            if(len(group.endpoint) == max_time):
                temp = pd.DataFrame(
                        {
                        'chemical.id': np.unique(temp_df['chemical.id']),
                        'plate.id': np.unique(temp_df['plate.id']),
                        'well': np.unique(temp_df['well']),
                        'chemical_plate_well': np.unique(temp_df['chemical_plate_well']),
                        'conc': np.unique(temp_df['conc'])
                        })
                #print(temp.head())
                # Append additonal columns corresponding to time points
                for time_point in np.arange(max_time):
                    end_point = 't'+ str(time_point+1)
                    temp = pd.concat([temp, pd.DataFrame({end_point: temp_df.value[temp_df.endpoint == end_point].values})],axis = 1)
                #print(temp.head())
                #print(reformatted_data)
                reformatted_data = pd.concat([reformatted_data, temp])
                #print(reformatted_data)
end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("Done, it took:"+str(time_took)) 
# took 106 seconds for all 7 chemicals

chemical_index:3756


,chemical.id,conc,plate.id,well,endpoint,value
1152,3756,0.0,20544,H01,t1,0.0
1153,3756,0.0,20544,H02,t1,0.0
1154,3756,0.0,20544,H03,t1,0.0
1155,3756,0.0,20544,H04,t1,0.0
1156,3756,0.0,20544,H05,t1,0.0


,chemical.id,conc,plate.id,well,endpoint,value
100219,3756,100.0,20625,A08,t50,0.0
100220,3756,100.0,20625,A09,t50,0.0
100221,3756,100.0,20625,A10,t50,0.0
100222,3756,100.0,20625,A11,t50,0.0
100223,3756,100.0,20625,A12,t50,0.0


unique_endpoint_per_chemical:['t1' 't10' 't11' 't12' 't13' 't14' 't15' 't16' 't17' 't18' 't19' 't2'
 't20' 't21' 't22' 't23' 't24' 't25' 't26' 't27' 't28' 't29' 't3' 't30'
 't31' 't32' 't33' 't34' 't35' 't36' 't37' 't38' 't39' 't4' 't40' 't41'
 't42' 't43' 't44' 't45' 't46' 't47' 't48' 't49' 't5' 't50' 't6' 't7' 't8'
 't9']
len(group.endpoint):50
chemical_index:3758


,chemical.id,conc,plate.id,well,endpoint,value
288,3758,0.0,21407,H01,t1,0.0
289,3758,0.0,21407,H02,t1,0.0
290,3758,0.0,21407,H03,t1,0.0
291,3758,0.0,21407,H04,t1,0.0
292,3758,0.0,21407,H05,t1,0.0


...

,chemical.id,conc,plate.id,well,endpoint,value
99067,3762,100.0,21295,A08,t50,0.0
99068,3762,100.0,21295,A09,t50,0.0
99069,3762,100.0,21295,A10,t50,0.0
99070,3762,100.0,21295,A11,t50,0.0
99071,3762,100.0,21295,A12,t50,0.0


unique_endpoint_per_chemical:['t1' 't10' 't11' 't12' 't13' 't14' 't15' 't16' 't17' 't18' 't19' 't2'
 't20' 't21' 't22' 't23' 't24' 't25' 't26' 't27' 't28' 't29' 't3' 't30'
 't31' 't32' 't33' 't34' 't35' 't36' 't37' 't38' 't39' 't4' 't40' 't41'
 't42' 't43' 't44' 't45' 't46' 't47' 't48' 't49' 't5' 't50' 't6' 't7' 't8'
 't9']
len(group.endpoint):50
chemical_index:3859


,chemical.id,conc,plate.id,well,endpoint,value
1728,3859,0.0,20488,A01,t1,0.0
1729,3859,0.0,20488,A02,t1,0.0
1730,3859,0.0,20488,A03,t1,0.0
1731,3859,0.0,20488,A04,t1,0.0
1732,3859,0.0,20488,A05,t1,0.0


,chemical.id,conc,plate.id,well,endpoint,value
100795,3859,100.0,20491,H08,t50,0.0
100796,3859,100.0,20491,H09,t50,0.0
100797,3859,100.0,20491,H10,t50,0.0
100798,3859,100.0,20491,H11,t50,0.0
100799,3859,100.0,20491,H12,t50,0.0


unique_endpoint_per_chemical:['t1' 't10' 't11' 't12' 't13' 't14' 't15' 't16' 't17' 't18' 't19' 't2'
 't20' 't21' 't22' 't23' 't24' 't25' 't26' 't27' 't28' 't29' 't3' 't30'
 't31' 't32' 't33' 't34' 't35' 't36' 't37' 't38' 't39' 't4' 't40' 't41'
 't42' 't43' 't44' 't45' 't46' 't47' 't48' 't49' 't5' 't50' 't6' 't7' 't8'
 't9']
len(group.endpoint):50
Done, it took:106.6 seconds


In [12]:
display(reformatted_data)
unique_chemical_id_reformatted_data = np.unique(reformatted_data['chemical.id'])
print (unique_chemical_id_reformatted_data)

,chemical.id,plate.id,well,chemical_plate_well,conc,t1,t2,t3,t4,t5,...,t41,t42,t43,t44,t45,t46,t47,t48,t49,t50
0,3756,20544,A01,3756_20544_A01,100.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
0,3756,20544,A02,3756_20544_A02,100.0,0.0,0.000000,31.445283,0.000000,33.593737,...,12.890625,0.000000,19.422747,5.295147,0.0,0.0,0.0,0.0,0.0,9.765619
0,3756,20544,A03,3756_20544_A03,100.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
0,3756,20544,A04,3756_20544_A04,100.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
0,3756,20544,A05,3756_20544_A05,100.0,0.0,0.000000,0.000000,90.364532,23.893231,...,0.000000,28.103313,5.013020,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,3859,20491,H08,3859_20491_H08,100.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
0,3859,20491,H09,3859_20491_H09,100.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
0,3859,20491,H10,3859_20491_H10,100.0,0.0,0.000000,0.000000,8.159719,20.833312,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
0,3859,20491,H11,3859_20491_H11,100.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000


[3756 3758 3759 3760 3761 3762 3859]


In [13]:
reformatted_data_filename = str(complete_input_file_path[:-4]) + "_wide_t1_t50_" + str(full_devel) + ".csv"
reformatted_data.to_csv(reformatted_data_filename,index=False)